# 1️⃣ One-Time Setup (Run Once)

### Create Database & Tables

In [0]:
#### Previous test

# Set up the Spark configuration for ADLS access
spark.conf.set(
    "fs.azure.sas.poddemo.demolakehouse.blob.core.windows.net",
    "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-03-24T12:27:36Z&st=2025-03-24T04:27:36Z&spr=https&sig=SWuoDnXSaD7CkcIYirThAH11YTJmQvqFvKODe908mDc%3D"
)

# Define Catalog & Database
catalog_name = "podlakehouse"
database_name = "Aqualake"

# Create Database (if not exists)
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{database_name}")
print(f"✅ Database {catalog_name}.{database_name} is ready!")


✅ Database podlakehousedemo.Aqualake is ready!


In [0]:
# Set up the Spark configuration for ADLS access
spark.conf.set(
    "fs.azure.sas.demolakehouse.poddemo.blob.core.windows.net",
    "sp=r&st=2025-05-16T12:11:33Z&se=2025-05-16T20:11:33Z&spr=https&sv=2024-11-04&sr=c&sig=gfDjzKmnYykJR97RAscAylJBNh%2BpHjOSGzOXuvHC02w%3D"
)

# Define Catalog & Database
catalog_name = "podlakehouse"
database_name = "Aqualake"

# Create Database (if not exists)
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_name}.{database_name}")
print(f"✅ Database {catalog_name}.{database_name} is ready!")


✅ Database podlakehouse.Aqualake is ready!


### Create Control Framework Table

In [0]:
# Create Control Framework Table (One-time setup)
control_table = f"{catalog_name}.{database_name}.control_framework"

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {control_table} (
    TableName STRING,
    SourceCount BIGINT,
    TargetCount BIGINT,
    LoadTime TIMESTAMP,
    Status STRING
) USING DELTA;
""")

print("✅ Control Framework Table is ready!")


✅ Control Framework Table is ready!


# 2️⃣ Repeated Data Load (Run Multiple Times)

### Step 1: Read CSV Files from ADLS

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, lit

# Define ADLS Source Path
adls_path = "wasbs://demolakehouse@poddemo.blob.core.windows.net/"


# Expected files with their table names
expected_files = {
    "customers": "Customers.csv",
    "sales": "Sales.csv",
    "categories": "categories.csv",
    "cities": "cities.csv",
    "countries": "countries.csv",
    "employees": "employees.csv",
    "products": "products.csv"
}

# Dictionary to store DataFrames
dfs = {}

# Read CSV files dynamically
for table_name, file_name in expected_files.items():
    file_path = adls_path + file_name
    try:
        df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)
        dfs[table_name] = df
        print(f"✅ Successfully loaded {file_name} into {table_name}_df")
    except Exception as e:
        print(f"⚠️ Warning: {file_name} not found. Skipping...")

# Show available DataFrames
for table_name, df in dfs.items():
    print(f"📌 Sample data from {table_name.upper()}:")
    display(df.limit(5))


✅ Successfully loaded Customers.csv into customers_df
✅ Successfully loaded Sales.csv into sales_df
✅ Successfully loaded categories.csv into categories_df
✅ Successfully loaded cities.csv into cities_df
✅ Successfully loaded countries.csv into countries_df
✅ Successfully loaded employees.csv into employees_df
✅ Successfully loaded products.csv into products_df
📌 Sample data from CUSTOMERS:
+----------+---------+-------------+--------+------+--------------------+
|CustomerID|FirstName|MiddleInitial|LastName|CityID|             Address|
+----------+---------+-------------+--------+------+--------------------+
|         1| Stefanie|            Y|    Frye|    79|       97 Oak Avenue|
|         2|    Sandy|            T|   Kirby|    96|52 White First Fr...|
|         3|      Lee|            T|   Zhang|    55|921 White Fabien ...|
|         4|   Regina|            S|   Avery|    40|       75 Old Avenue|
|         5|   Daniel|            S|  Mccann|     2|283 South Green H...|
+----------+--

### Step 2: Enrich Data

In [0]:
from pyspark.sql.functions import current_timestamp, lit

# Add Timestamp & ActionStatus Columns
dfs_enriched = {}
for table_name, df in dfs.items():
    df = df.withColumn("Timestamp", current_timestamp()).withColumn("ActionStatus", lit("Insert"))
    dfs_enriched[table_name] = df
    print(f"✅ Added Timestamp & ActionStatus to {table_name}")


✅ Added Timestamp & ActionStatus to customers
✅ Added Timestamp & ActionStatus to sales
✅ Added Timestamp & ActionStatus to categories
✅ Added Timestamp & ActionStatus to cities
✅ Added Timestamp & ActionStatus to countries
✅ Added Timestamp & ActionStatus to employees
✅ Added Timestamp & ActionStatus to products


### Step 3: Perform UPSERT (MERGE INTO Delta Tables)

In [0]:
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
from pyspark.sql import Row

# Define Primary Keys for Tables
primary_keys = {
    "customers": "CustomerID",
    "sales": "SalesID",
    "categories": "CategoryID",
    "cities": "CityID",
    "countries": "CountryID",
    "employees": "EmployeeID",
    "products": "ProductID"
}

# Schema for Control Framework
control_schema = StructType([
    StructField("TableName", StringType(), False),
    StructField("SourceCount", LongType(), False),
    StructField("TargetCount", LongType(), False),
    StructField("LoadTime", TimestampType(), False),
    StructField("Status", StringType(), False)
])

# Initialize Tracking List
control_records = []

# Process Each Table
for table_name, df in dfs_enriched.items():
    full_table_name = f"{catalog_name}.{database_name}.{table_name}"

    # Count source records
    source_count = df.count()

    try:
        # Check if Delta Table Exists
        if spark._jsparkSession.catalog().tableExists(full_table_name):
            print(f"🔄 Performing UPSERT (MERGE) for {table_name}...")

            # Dynamically get column names
            columns = [col for col in df.columns if col not in ["ActionStatus", "Timestamp"]]

            # Construct Update Clause
            update_set_clause = ", ".join([f"target.{col} = source.{col}" for col in columns])

            merge_query = f"""
            MERGE INTO {full_table_name} AS target
            USING (SELECT * FROM new_data) AS source
            ON target.{primary_keys[table_name]} = source.{primary_keys[table_name]}
            WHEN MATCHED THEN 
                UPDATE SET 
                    {update_set_clause}, 
                    target.ActionStatus = 'Update', 
                    target.Timestamp = current_timestamp()
            WHEN NOT MATCHED THEN 
                INSERT ({", ".join(df.columns)}) 
                VALUES ({", ".join(["source." + col for col in df.columns])})
            """

            # Create Temporary View
            df.createOrReplaceTempView("new_data")
            spark.sql(merge_query)

        else:
            print(f"🚀 First Load: Creating Delta Table for {table_name}...")
            df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)

        # Count Target Records After Load
        target_count = spark.read.table(full_table_name).count()

        # Log Success in Control Framework
        control_records.append(Row(table_name, int(source_count), int(target_count), datetime.now(), "Success"))

        print(f"✅ {table_name} loaded successfully: Source ({source_count}) → Target ({target_count})")

    except Exception as e:
        print(f"❌ Error loading {table_name}: {str(e)}")
        control_records.append(Row(table_name, int(source_count), 0, datetime.now(), "Failure"))


🚀 First Load: Creating Delta Table for customers...
✅ customers loaded successfully: Source (2000) → Target (2000)
🚀 First Load: Creating Delta Table for sales...
✅ sales loaded successfully: Source (12556) → Target (12556)
🚀 First Load: Creating Delta Table for categories...
✅ categories loaded successfully: Source (11) → Target (11)
🚀 First Load: Creating Delta Table for cities...
✅ cities loaded successfully: Source (96) → Target (96)
🚀 First Load: Creating Delta Table for countries...
✅ countries loaded successfully: Source (206) → Target (206)
🚀 First Load: Creating Delta Table for employees...
✅ employees loaded successfully: Source (23) → Target (23)
🚀 First Load: Creating Delta Table for products...
✅ products loaded successfully: Source (452) → Target (452)


### Step 4: Update Control Framework

In [0]:
# Convert Tracking Data to DataFrame
control_df = spark.createDataFrame(control_records, schema=control_schema)

# Define Control Framework Table
control_table = f"{catalog_name}.{database_name}.control_framework"

# Append Control Records
if spark._jsparkSession.catalog().tableExists(control_table):
    print("🔎 Appending to Control Framework...")
    control_df.write.mode("append").saveAsTable(control_table)
else:
    print("🚀 Creating Control Framework Table...")
    control_df.write.format("delta").mode("overwrite").saveAsTable(control_table)

print("📊 Control Framework updated successfully!")


🔎 Appending to Control Framework...
📊 Control Framework updated successfully!


In [0]:
%sql
select * from aqualake.control_framework

TableName,SourceCount,TargetCount,LoadTime,Status
employees,23,23,2025-05-16T12:27:29.064811Z,Success
products,452,452,2025-05-16T12:27:33.13956Z,Success
cities,96,96,2025-05-16T12:27:20.077753Z,Success
countries,206,206,2025-05-16T12:27:24.618449Z,Success
sales,12556,12556,2025-05-16T12:27:11.385704Z,Success
categories,11,11,2025-05-16T12:27:15.865612Z,Success
customers,2000,2000,2025-05-16T12:27:05.322807Z,Success


## Rough Script for reference

In [0]:
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
from pyspark.sql import Row

# Define Primary Keys for Tables
primary_keys = {
    "customers": "CustomerID",
    "sales": "SalesID",
    "categories": "CategoryID",
    "cities": "CityID",
    "countries": "CountryID",
    "employees": "EmployeeID",
    "products": "ProductID"
}

# Schema for Control Framework
control_schema = StructType([
    StructField("TableName", StringType(), False),
    StructField("SourceCount", LongType(), False),
    StructField("TargetCount", LongType(), False),
    StructField("LoadTime", TimestampType(), False),
    StructField("Status", StringType(), False)
])

# Initialize Tracking List
control_records = []

# Process Each Table
for table_name, df in dfs_enriched.items():
    full_table_name = f"{catalog_name}.{database_name}.{table_name}"

    # Count source records
    source_count = df.count()

    try:
        # Check if Delta Table Exists (Improved Method)
        if spark.catalog._jcatalog.tableExists(full_table_name):
            print(f"🔄 Performing UPSERT (MERGE) for {table_name}...")

            # Dynamically get column names excluding ActionStatus and Timestamp
            columns = [col for col in df.columns if col not in ["ActionStatus", "Timestamp"]]

            # Construct Update Clause
            update_set_clause = ", ".join([f"target.{col} = source.{col}" for col in columns])

            # Create a Temporary View for the Source Data
            df.createOrReplaceTempView("new_data")

            # Merge Query (Fixed USING Clause)
            merge_query = f"""
            MERGE INTO {full_table_name} AS target
            USING new_data AS source
            ON target.{primary_keys[table_name]} = source.{primary_keys[table_name]}
            WHEN MATCHED THEN 
                UPDATE SET 
                    {update_set_clause}, 
                    target.ActionStatus = 'Update', 
                    target.Timestamp = current_timestamp()
            WHEN NOT MATCHED THEN 
                INSERT ({", ".join(df.columns)}) 
                VALUES ({", ".join(df.columns)})
            """

            # Execute the Merge Query
            spark.sql(merge_query)

        else:
            print(f"🚀 First Load: Creating Delta Table for {table_name}...")
            df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)

        # Count Target Records After Load
        target_count = spark.read.table(full_table_name).count()

        # Log Success in Control Framework
        control_records.append(Row(table_name, int(source_count), int(target_count), datetime.now(), "Success"))

        print(f"✅ {table_name} loaded successfully: Source ({source_count}) → Target ({target_count})")

    except Exception as e:
        print(f"❌ Error loading {table_name}: {str(e)}")
        control_records.append(Row(table_name, int(source_count), 0, datetime.now(), "Failure"))
